<a href="https://colab.research.google.com/github/ryu-0729/ml-python/blob/master/chapter3/horse_or_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 馬と人間を判定するモデル

In [2]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2023-02-25 16:15:33--  https://storage.googleapis.com/learning-datasets/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  19.5MB/s    in 9.0s    

2023-02-25 16:15:43 (15.9 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2023-02-25 16:15:47--  https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  7.16MB/s    in 1.5s    

2023-02-25 16:15:49 (7.16 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



#### 学習用

In [5]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
zip_ref.close()

In [6]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

In [7]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

['horse35-5.png', 'horse07-4.png', 'horse12-1.png', 'horse41-6.png', 'horse10-2.png', 'horse21-8.png', 'horse17-4.png', 'horse45-9.png', 'horse10-1.png', 'horse25-8.png']
['human12-12.png', 'human10-29.png', 'human04-19.png', 'human08-10.png', 'human11-14.png', 'human09-29.png', 'human01-30.png', 'human07-29.png', 'human12-09.png', 'human03-14.png']


#### 検証用

In [8]:
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [9]:
# Directory with our validation horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our validation human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [10]:
validation_horse_names = os.listdir(validation_horse_dir)
print(validation_horse_names[:10])

validation_human_names = os.listdir(validation_human_dir)
print(validation_human_names[:10])

['horse3-484.png', 'horse5-303.png', 'horse1-554.png', 'horse5-076.png', 'horse5-589.png', 'horse2-383.png', 'horse2-269.png', 'horse1-122.png', 'horse4-599.png', 'horse5-032.png']
['valhuman04-20.png', 'valhuman01-17.png', 'valhuman04-12.png', 'valhuman05-03.png', 'valhuman03-17.png', 'valhuman05-19.png', 'valhuman03-22.png', 'valhuman01-05.png', 'valhuman05-02.png', 'valhuman03-05.png']


In [11]:
import tensorflow as tf

In [12]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [14]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=0.001),
    metrics=['accuracy'],
)

#### 画像にラベル付け
- 左または右に最大40度までのランダムな回転
- 最大20%の縦または横の移動
- 最大20%のせん断変形
- 最大20%の拡大
- ランダムに水平または垂直斑点
- 移動またはせん断後に欠落したピクセルを最近傍の値で重填

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# NOTE: 学習用
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1. /255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    '/tmp/horse-or-human/',  # This is the source directory for training images
    target_size=(300, 300),  # All images will be resized to 150x150
    batch_size=128,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary'
)


Found 1027 images belonging to 2 classes.


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# NOTE: 検証用
# All images will be rescaled by 1./255
validation_datagen = ImageDataGenerator(
    rescale=1. /255,
    # rotation_range=40,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest',
)

# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human/',  # This is the source directory for validation images
    target_size=(300, 300),  # All images will be resized to 150x150
    batch_size=128,
    # Since we use binary_crossentropy loss, we need binary labels
    class_mode='binary'
)


Found 256 images belonging to 2 classes.


#### モデルの学習

In [22]:
history = model.fit(
    train_generator,
    steps_per_epoch=8, 
    epochs=15,
    validation_data=validation_generator,
    # verbose=1,
)

Epoch 1/15
8/8 [==============================] - 25s 3s/step - loss: 0.7024 - accuracy: 0.8799 - val_loss: 2.2426 - val_accuracy: 0.5195
Epoch 2/15
8/8 [==============================] - 24s 3s/step - loss: 0.2640 - accuracy: 0.8988 - val_loss: 2.0085 - val_accuracy: 0.6211
Epoch 3/15
8/8 [==============================] - 24s 3s/step - loss: 0.3589 - accuracy: 0.8610 - val_loss: 1.2023 - val_accuracy: 0.6992
Epoch 4/15
8/8 [==============================] - 24s 3s/step - loss: 0.1449 - accuracy: 0.9433 - val_loss: 1.5385 - val_accuracy: 0.7188
Epoch 5/15
8/8 [==============================] - 23s 3s/step - loss: 0.1618 - accuracy: 0.9366 - val_loss: 1.8433 - val_accuracy: 0.7109
Epoch 6/15
8/8 [==============================] - 26s 3s/step - loss: 0.1364 - accuracy: 0.9463 - val_loss: 2.1623 - val_accuracy: 0.6758
Epoch 7/15
8/8 [==============================] - 24s 3s/step - loss: 0.3343 - accuracy: 0.8598 - val_loss: 1.4652 - val_accuracy: 0.5977
Epoch 8/15
8/8 [==================

#### モデルのテスト

In [23]:
import numpy as np
from google.colab import files
from keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving beautiful-gd005465f3_1920.jpg to beautiful-gd005465f3_1920.jpg
Saving woman-g9dde4100e_1920.jpg to woman-g9dde4100e_1920.jpg
Saving girl-g08a48d216_1920.jpg to girl-g08a48d216_1920 (2).jpg
Saving horses-g88e2feb8b_1920.jpg to horses-g88e2feb8b_1920 (2).jpg
Saving woman-g912731b77_1920.jpg to woman-g912731b77_1920 (2).jpg
1/1 [==============================] - 0s 20ms/step
[0.]
beautiful-gd005465f3_1920.jpg is a horse
1/1 [==============================] - 0s 20ms/step
[1.]
woman-g9dde4100e_1920.jpg is a human
1/1 [==============================] - 0s 19ms/step
[1.]
girl-g08a48d216_1920.jpg is a human
1/1 [==============================] - 0s 19ms/step
[0.]
horses-g88e2feb8b_1920.jpg is a horse
1/1 [==============================] - 0s 20ms/step
[0.]
woman-g912731b77_1920.jpg is a horse
